In [2]:
import pandas as pd
import unicodedata
from rutas import CSV_BIBLIOTECAS, CSV_CINES, CSV_MUSEOS


### 2 Procesamiento de Datos
<span>Normalizar toda la información de Museos, Salas de Cine y Bibliotecas Populares, para crear una única tabla que contenga:</span>

<style>
    ul{columns:4;}
    div{margin-top:25px;}
</style>
<div class='lista'>
    <ul>
        <li>cod_localidad</li>
        <li>id_provincia</li>
        <li>id_departamento</li>
        <li>categoría</li>
        <li>provincia</li>
        <li>localidad</li>
        <li>nombre</li>
        <li>domicilio</li>
        <li>código postal</li>
        <li>número de teléfono</li>
        <li>mail</li>
        <li>web</li>
    </ul>
</div>


#### 2.1 Normalizando y consolidando campos

In [3]:
# Leer csvs

raw_df_bibliotecas = pd.read_csv(CSV_BIBLIOTECAS)
raw_df_museos = pd.read_csv(CSV_MUSEOS)
raw_df_cines = pd.read_csv(CSV_CINES)

In [4]:
def normalizar_csv(csv_file:str):
    '''
    Carga un archivo csv como DataFrame y normaliza tanto sus campos
    como sus valores tipo string.
    Toma como argumento un string con la ruta al CSV
    '''
    # Leer csv
    raw_df = pd.read_csv(csv_file)
    # Normalizar campos
    raw_df.columns = raw_df.columns.str.lower()
    raw_df.columns = raw_df.columns.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')
    
    # Normalizar datos
    raw_df = raw_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    raw_df.fillna('s/d', inplace=True)

    def no_tildes(texto:str):
        '''
        Quita las tildes de un string. Toma como argumento un string.
        '''
        new_value = unicodedata.normalize('NFKD',texto).encode('ascii',errors='ignore').decode('utf-8')
        return new_value
    
    new_df = raw_df.applymap(lambda x: no_tildes(x) if isinstance(x, str) else x)
    
    # Unificar Tierra Del Fuego
    def limpiar_tdf(provincia:pd.Series):
        '''
        Quita el exceso de texto para 'Tierra del Fuego' en el campo 'Provincia'.
        Toma como argumento un objeto Pandas Series.
        '''
        return provincia.split(',')[0]

    new_df['provincia'] = new_df['provincia'].apply(limpiar_tdf)
    return new_df

raw_df_bibliotecas = normalizar_csv(CSV_BIBLIOTECAS)
raw_df_museos = normalizar_csv(CSV_MUSEOS)
raw_df_cines = normalizar_csv(CSV_CINES)

In [12]:
def nuevos_campos(df:pd.DataFrame):
    match df.categoria[0]:
        case 'Salas de cine':
            new_df = df[['cod_localidad', 'id_provincia', 'id_departamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp']].copy()
            new_df[['número de teléfono', 'mail']] = 's/d'
            new_df['web'] = df['web']
        
        case 'Espacios de Exhibicion Patrimonial':
            new_df = df[['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp', 'telefono', 'mail', 'web']]
        
        case 'Bibliotecas Populares':
            new_df = df[['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'domicilio', 'cp', 'telefono', 'mail', 'web']]
    
    return new_df

In [15]:
# Seleccionar columnas requeridas en cada dataframe
df_bibliotecas = nuevos_campos(raw_df_bibliotecas)
df_museos = nuevos_campos(raw_df_museos)
df_cines = nuevos_campos(raw_df_cines)

In [16]:
# Renombrando columnas
def rename_cols(df):
    '''
    Crea un diccionario a partir de un zip entre las columnas actuales 
    del dataframe y una lista con sus correspondientes nuevos nombres
    Pasar como argumento el dataframe con las columnas a renombrar
    '''
    # Crear lista de nomenclaturas para nueva tabla
    columnas = ['cod_localidad', 'id_provincia', 'id_departamento', 'categoría', 
                'provincia', 'localidad', 'nombre', 'domicilio', 'código postal', 
                'número de teléfono', 'mail', 'web']
    diccionario = dict(zip(df.columns, columnas))
    df = df.rename(columns=diccionario)
    return df


In [17]:
# Crear dataframes con columnas renombradas
df_bibliotecas = rename_cols(df_bibliotecas)
df_museos = rename_cols(df_museos)
df_cines = rename_cols(df_cines)

#### 2.2 Normalizando datos

In [9]:
df_bibliotecas.fillna('s/d', inplace=True)
df_museos.fillna('s/d', inplace=True)
df_cines.fillna('s/d', inplace=True)

### 3 Creación de Tablas

#### 3.1 Crear tabla única

In [10]:
consolidado = pd.concat([df_bibliotecas,df_museos,df_cines], ignore_index=True)

#### 3.2 Crear tabla de cantidad de registros

In [19]:
contador_registros = pd.concat([raw_df_bibliotecas,raw_df_museos,raw_df_cines], ignore_index=True)[['provincia','categoria','fuente']]

#### 3.3 Crear tabla de cines

In [23]:
contador_cines = raw_df_cines[['provincia','pantallas','butacas','espacio_incaa']]